In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import torch
import re
import sys
import wiki_article 
import glob
import modeling
from dataclasses import dataclass
from io import StringIO
from transformers import AutoModelWithLMHead, AutoTokenizer
from scipy import stats

In [ ]:
import r

In [305]:
interleave = 768
wiki_article.generate_text_dataset(
    open("wikitext-2-raw/wiki.train.raw", "r"), 
    open("wikitext_title_train/wiki_title.train.raw", "w"),
    interleave=interleave,
)
wiki_article.generate_text_dataset(
    open("wikitext-2-raw/wiki.valid.raw", "r"), 
    open("wikitext_title_train/wiki_title.valid.raw", "w"),
    interleave=interleave,
)

In [6]:
def get_checkpoints(base_dir):
    checkpoint_dirs = glob.glob(f"{base_dir}/checkpoint*")
    checkpoint_dirs.sort(key=lambda x: int(x[(x.index("checkpoint-") + len("checkpoint-")):]))
    return checkpoint_dirs
modeling_gpt
def evaluate_lm_checkpoints(base_dir, validation_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    for d in get_checkpoints(base_dir):
        model = AutoModelWithLMHead.from_pretrained(d).to('cuda')
        refined_model_eval = wiki_article.lm_eval(model, tokenizer, validation_path)
        print(f"{d}: {refined_model_eval}")

def evaluate_title_checkpoints(base_dir, validation_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    for d in get_checkpoints(base_dir):
        model = AutoModelWithLMHead.from_pretrained(d).to('cuda')
        refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, validation_path)
        print(f"{d}: m={refined_model_eval.mean}, v={refined_model_eval.variance}")

# evaluate_lm_checkpoints("models/wikitext_103_stride_512_v0/", "data/wikitext-103-title-train/wiki_title.valid.raw")
#print(glob.glob("models/wikitext_103_stride_512_v0/*"))

models/wikitext_103_stride_512_v0/checkpoint-500: 24.66460609436035
models/wikitext_103_stride_512_v0/checkpoint-1000: 23.94955062866211
models/wikitext_103_stride_512_v0/checkpoint-1500: 22.988603591918945
models/wikitext_103_stride_512_v0/checkpoint-2000: 22.5709285736084
models/wikitext_103_stride_512_v0/checkpoint-2500: 22.404022216796875
models/wikitext_103_stride_512_v0/checkpoint-3000: 21.691320419311523
models/wikitext_103_stride_512_v0/checkpoint-3500: 21.813074111938477
models/wikitext_103_stride_512_v0/checkpoint-4000: 21.99334716796875
models/wikitext_103_stride_512_v0/checkpoint-4500: 21.768508911132812
models/wikitext_103_stride_512_v0/checkpoint-5000: 21.2707462310791
models/wikitext_103_stride_512_v0/checkpoint-5500: 21.388479232788086
models/wikitext_103_stride_512_v0/checkpoint-6000: 21.18572998046875
models/wikitext_103_stride_512_v0/checkpoint-6500: 21.76972770690918
models/wikitext_103_stride_512_v0/checkpoint-7000: 21.52361488342285
models/wikitext_103_stride_512_

KeyboardInterrupt: 

In [27]:

model = AutoModelWithLMHead.from_pretrained("gpt2").to('cuda')

In [9]:
unrefined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
unrefined_model_eval

DescribeResult(nobs=60, minmax=(9.851899147033691, 15.262736320495605), mean=12.927889172236124, variance=1.572701685821396, skewness=-0.44557272655893687, kurtosis=-0.35688076179019035)

In [30]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("output_103/").to('cuda')

In [17]:
refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
refined_model_eval

DescribeResult(nobs=60, minmax=(10.398584365844727, 17.0587158203125), mean=13.77767310142517, variance=1.9294105073945371, skewness=-0.01741720699601202, kurtosis=-0.0981178051223095)

In [102]:
sequence = f"\"TITLE\" is a song collaboration by Chinese artist Pamela Chen and Canadian singer Thomas Dimson, first released independently in March 2020. After gaining popularity amongst the cat community, the single was re-released by major label Columbia Records in May 2020. Pamela describes the song as being originally inspired by her two kittens, Apollo and Bean who once said meow.<bot>"

input = tokenizer.encode(sequence, return_tensors="pt").to('cuda')
generated = model.generate(input, max_length=100, num_return_sequences=10, temperature=1)

print(f"Prompt text: {sequence}")
for i in range(generated.size()[0]):
    sentence_tokens = generated[i, :].tolist()
    decoded = tokenizer.decode(sentence_tokens)
    m = re.search(r"<bot>(.modeling_gpt*?)<eot>", decoded)
    if m:
        print(f"{i}) {m.groups(1)}")
    else:
        print(f"{i}) Didn't work")
    

resulting_string = tokenizer.decode(generated.tolist()[0])
# print(resulting_string)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Prompt text: "TITLE" is a song collaboration by Chinese artist Pamela Chen and Canadian singer Thomas Dimson, first released independently in March 2020. After gaining popularity amongst the cat community, the single was re-released by major label Columbia Records in May 2020. Pamela describes the song as being originally inspired by her two kittens, Apollo and Bean who once said meow.<bot>
0) ('Animal Heart',)
1) ('The Song of a Cuddle',)
2) ('Letting You Come',)
3) ('Escape from Cats',)
4) ("So You Think It's All Good",)
5) ("Don 't Worry About You",)
6) ('I Am... TITLE',)
7) ('I Put a Smile on a Smile Baby',)
8) ('I Wanna Be',)
9) ('Pump the Pump',)


In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = modeling.GPT2LMHeadWithWeightedLossModel.from_pretrained("gpt2").to('cuda')

In [39]:
loss_weights = torch.tensor(
    [
        [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        [1.0, 1.0, 0.7, 1.0, 0.0, 1.0],
        [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    ]
).to('cuda')
wiki_article.perplexity(model, tokenizer, ["abc", "abc", "bbq", "abc abc really long sentence"], loss_weights=loss_weights)

tensor(-6.2348, device='cuda:0')

In [45]:
tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids("abc"))

39305

In [57]:
token_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("some text"))
token_title = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("<title>fun city</title>"))
wiki_article.ArticleTitleDataset._make_example(tokenizer, token_text, token_title)

([11246, 2420, 27, 7839, 29, 12543, 1748, 3556, 7839, 29, 124314],
 [False, False, False, True, True, True, True, True, True, True, False])